# Imports

In [ ]:
import pandas as pd

# import py files
from Classes import Player, Team, LeagueIndividual
from GA_mutation import mutation_swap_players, mutation_regenerate_team

# Local Data

In [ ]:
df = pd.read_csv("data/players.csv")
df = df.drop(columns=['Unnamed: 0']) # drop the index column

# Problem Configuration

In [ ]:
TEAM_SIZE = 7
NUM_TEAMS = 5
BUDGET_LIMIT = 750
TEAM_STRUCTURE = {"GK": 1, "DEF": 2, "MID": 2, "FWD": 2}
POPULATION_SIZE = 10

# Convert DF to player objects

In [ ]:
players_by_position = {
    pos: [Player.from_dict(row) for _, row in df[df['Position'] == pos].iterrows()]
    for pos in TEAM_STRUCTURE
}

# Generate Population

In [ ]:
# === GENERATE POPULATION ===
def generate_initial_population(size, players_by_position, team_structure, budget_limit, num_teams):
    population = []
    attempts = 0
    max_attempts = 1000 # avoid infinite loop if unable to generate valid leagues

    while len(population) < size and attempts < max_attempts:
        indiv = LeagueIndividual(players_by_position, team_structure, budget_limit, num_teams)
        if indiv.league is not None:
            population.append(indiv)
        attempts += 1

    return population

population = generate_initial_population(
    POPULATION_SIZE,
    players_by_position,
    TEAM_STRUCTURE,
    BUDGET_LIMIT, 
    NUM_TEAMS
)

# Check if Classes are working

In [ ]:
# === EXAMPLE USAGE ===
individual = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT, NUM_TEAMS)

# Print result
print("\n=== One League Example ===")
for i, team in enumerate(individual.league):
    print(f"\n🏆 Team {i + 1}")
    print(team)
    print(f"Avg Skill: {team.avg_skill():.2f} | Total Salary: €{team.total_salary()}M")

print(f"\nFitness: {individual.fitness:.4f}")

In [ ]:


# === PRINT POPULATION DETAILS ===
for idx, indiv in enumerate(population):
    print("\n" + "=" * 35)
    print(f"🏟️  League (Individual) {idx + 1}")
    print("=" * 35)

    for tidx, team in enumerate(indiv.league):
        print(f"\n  🏆 Team {tidx + 1}")
        print(team)
        print(f"    📊 Avg Skill: {team.avg_skill():.2f}")
        print(f"    💰 Total Salary: €{team.total_salary()}M")

    print(f"\n  ➤ League Fitness (Std Dev of team avg skill): {indiv.fitness:.4f}")


# Mutations

## Swap Players Betweeen Teams

In [ ]:
swap_players = mutation_swap_players(individual)
print(swap_players)

In [ ]:
if swap_players == individual:
    print("⚠️  No mutation applied (swap failed after multiple attempts).")
else:
    print("✅ Mutation applied successfully.")

In [ ]:
# # test mutation_swap_players
# 
# def print_league_details(league_indiv, label=""):
#     print(f"\n{'='*50}\n🏟️  {label} (Fitness: {league_indiv.fitness:.4f})\n{'='*50}")
#     
#     all_players = set()
#     valid = True
# 
#     for i, team in enumerate(league_indiv.league):
#         print(f"\n🏆 Team {i+1}")
#         for player in team.players:
#             print(f"  - {player}")
#         avg = team.avg_skill()
#         total_salary = team.total_salary()
#         print(f"    📊 Avg Skill: {avg:.2f} | 💰 Total Salary: €{total_salary}M")
#         
#         all_players.update(player.name for player in team.players)
# 
#         if not team.is_valid(league_indiv.team_structure, league_indiv.budget_limit):
#             print("    ❌ Invalid team (structure or budget)")
#             valid = False
# 
#     expected_total = sum(league_indiv.team_structure.values()) * len(league_indiv.league)
#     if len(all_players) != expected_total:
#         print("⚠️  Duplicate players found across teams!")
#         valid = False
# 
#     print(f"\n✅ Valid League: {valid}")
#     return valid
# 
# 
# # === GENERATE ONE INDIVIDUAL AND MUTATE IT ===
# original = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT, NUM_TEAMS)
# mutated = mutation_swap_players(original)
# 
# # === PRINT RESULTS ===
# print_league_details(original, "Original League")
# print_league_details(mutated, "Mutated League")


In [ ]:
regenerate_team, success = mutation_regenerate_team(individual)
print(regenerate_team)

In [ ]:
if regenerate_team == individual:
    print("⚠️  No mutation applied (regenerate team failed).")
else:
    print("✅ Mutation applied successfully.")

In [ ]:
# def print_player_diff(orig_players, new_players):
#     orig_names = set(p.name for p in orig_players)
#     new_names = set(p.name for p in new_players)
#     added = new_names - orig_names
#     removed = orig_names - new_names
#     return added, removed
#
# def print_detailed_comparison(original, mutated):
#     print(f"\n{'='*60}")
#     print(f"🏟️  Regenerate Team Mutation Comparison")
#     print(f"📈 Fitness: {original.fitness:.4f} → {mutated.fitness:.4f}")
#     print(f"{'='*60}\n")
#
#     # === First Pass: Determine the regenerated team ===
#     max_changes = 0
#     regenerated_team_index = None
#     for i, (team_orig, team_mut) in enumerate(zip(original.league, mutated.league)):
#         orig_names = set(p.name for p in team_orig.players)
#         new_names = set(p.name for p in team_mut.players)
#         changes = len(orig_names.symmetric_difference(new_names))
#         if changes > max_changes:
#             max_changes = changes
#             regenerated_team_index = i
#
#     # === Second Pass: Print teams ===
#     for i, (team_orig, team_mut) in enumerate(zip(original.league, mutated.league)):
#         team_label = f"🏆 Team {i+1}"
#         if i == regenerated_team_index:
#             team_label += " 🆕"
#         print(f"\n{team_label}")
#
#         print(f"Original Team:")
#         for p in team_orig.players:
#             print(f"  - {p}")
#         print(f"    📊 Avg Skill: {team_orig.avg_skill():.2f} | 💰 Salary: €{team_orig.total_salary()}M")
#
#         print(f"Mutated Team:")
#         for p in team_mut.players:
#             print(f"  - {p}")
#         print(f"    📊 Avg Skill: {team_mut.avg_skill():.2f} | 💰 Salary: €{team_mut.total_salary()}M")
#
#         added, removed = print_player_diff(team_orig.players, team_mut.players)
#         if added or removed:
#             print(f"    ➕ Added: {', '.join(added) if added else 'None'}")
#             print(f"    ➖ Removed: {', '.join(removed) if removed else 'None'}")
#
#     print(f"\n✅ League Valid: {all(team.is_valid(mutated.team_structure, mutated.budget_limit) for team in mutated.league)}")
#     print(f"📌 Regenerated Team: Team {regenerated_team_index + 1 if regenerated_team_index is not None else 'Unknown'}")

# # === GENERATE ONE INDIVIDUAL AND MUTATE IT ===
# original = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT, NUM_TEAMS)
# mutated, success = mutation_regenerate_team(original)
#
# if success:
#     print_detailed_comparison(original, mutated)
# else:
#     print("⚠️ Mutation failed (could not perform valid regeneration)")